# Velocity Packet Tracker Visualisation

A file to generate a SDEC plot. And to a second plot showing the abundance of each element as a function of velocity

## Atomic Data

Using the [kurucz_cd23_chianti_H_He.h5](https://dev.azure.com/tardis-sn/TARDIS/_apis/git/repositories/tardis-refdata/items?path=atom_data/kurucz_cd23_chianti_H_He.h5&resolveLfs=true) dataset, as recommended

In [1]:
from tardis.io.atom_data.util import download_atom_data

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
download_atom_data('kurucz_cd23_chianti_H_He')

## Example Configuration File

The configuration file [tardis_example.yml](https://github.com/tardis-sn/tardis/tree/master/docs/tardis_example.yml) is used throughout this

In [3]:
!wget -q -nc https://raw.githubusercontent.com/tardis-sn/tardis/master/docs/tardis_example.yml

In [ ]:
!cat tardis_example.yml

## Running the Simulation

Importing the `run_tardis` function and creating the `sim` object

In [5]:
from tardis import run_tardis

In [ ]:
sim = run_tardis("tardis_example.yml", 
                 virtual_packet_logging=True,
                 show_convergence_plots=True,
                 export_convergence_plots=True,
                 log_level="INFO") 

## Generating SDEC Plot

Finally, generating the SDEC plot with `matplotlib`

In [7]:
from tardis.visualization import SDECPlotter

In [8]:
import matplotlib.pyplot as plt

In [ ]:
# Create plot
plot = SDECPlotter.from_simulation(sim)
fig = plot.generate_plot_mpl()

# Customize plot
fig.set_facecolor('#f5f5f5')  # Set background color

## Visualising Abundance

Relating abundance of the elements against velocity

In [11]:
import matplotlib.pyplot as plt

### Define Variables

In [12]:
# Converting velocity to km/s
vel_of_shell = (sim.model.v_inner / 1000).to('km/s')

abundance = sim.plasma.abundance

# Converting density to g/cm^3
density_of_shell = (sim.model.homologous_density.density_0 / 1000**3).to('g/cm^3')

### Plot Fractional Abudances

In [ ]:
fig, abundance_axis = plt.subplots(figsize=(20, 5))
atom_data = sim.plasma.atomic_data.atom_data

for atomic_number in sim.plasma.selected_atoms:
    atom_abundance = abundance.loc[atomic_number]
    abundance_axis.step(vel_of_shell, atom_abundance, marker='.', label=atom_data.symbol.loc[atomic_number])

abundance_axis.set(title='Trend of Elements vs Velocity', xlabel='Inner Velocity (km/s)', ylabel='Abundance (Fractional)')

density_axis = abundance_axis.twinx()
density_axis.step(vel_of_shell, density_of_shell[1:], marker='s', label='Density')
density_axis.set(ylabel='Density (g/cm^3)')

_ = fig.legend()

### Density vs Velocity for Different Elements

This plot shows the fractional abundance of different elements as a function of inner velocity, with the density of the shell overlaid as a black dashed line on the right axis. Each element is represented by a different colored line, with a legend indicating the symbol for each element

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

for atomic_number in sim.plasma.selected_atoms:
    atom_abundance = abundance.loc[atomic_number]
    ax.plot(vel_of_shell, atom_abundance, label=atom_data.symbol.loc[atomic_number])
    
ax.set(xlabel='Inner Velocity in Metre/Sec', ylabel='Abundance (Fractional)', title='Trend of Elements vs Velocity')
ax.legend()

ax2 = ax.twinx()
ax2.plot(vel_of_shell, density_of_shell[1:], 'k--', label='Density')
ax2.set(ylabel='Density in Kg/Cubic Metre')

fig.legend()
plt.show()


### Bubble Chart

This plot shows the density of each element at the outermost shell as a function of inner velocity using a bubble chart. Each element is represented by a circle with a size proportional to its fractional abundance at the outermost shell, and a color indicating its atomic symbol. The size and position of the outermost shell circle is fixed, while the position of the element circles varies with velocity. This plot is useful for comparing the densities of different elements at the outermost shell across a range of velocities

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

abundance_data = []
density_data = []

for atomic_number in sim.plasma.selected_atoms:
    atom_abundance = abundance.loc[atomic_number].iloc[-1]
    atom_density = atom_abundance * density_of_shell[-1]
    abundance_data.append(atom_abundance)
    density_data.append(atom_density)

ax.scatter(vel_of_shell[-1], density_of_shell[-1], s=200, c='r', alpha=0.5, label='Outermost Shell')

for i, atomic_number in enumerate(sim.plasma.selected_atoms):
    ax.scatter(vel_of_shell[-1], density_data[i], s=500*abundance_data[i], alpha=0.5, label=atom_data.symbol.loc[atomic_number])

ax.set(xlabel='Inner Velocity in Metre/Sec', ylabel='Density in Kg/Cubic Metre', title='Elemental Densities vs Velocity')
ax.legend()

plt.show()
